In [1784]:
import pandas as pd
import numpy as np
import re
from datetime import timedelta


## 1. Pasos de la depuración

- Exportacion de csv a trabajar.
- Visualizacion de los datos transformandolos a un DataFrame y asi tener entendimiento del contenido a trabajar.
- Limpieza de datos por columna y verificacion de tipo de dato cada una tiene. 
- Se debe identificar errores: Detectar datos inválidos, incompletos, duplicados o irrelevantes.


- Exportación del fichero CSV y transformacion a un DataFrame 

In [1785]:
retail2_df = pd.read_csv('retail2.csv')

- Reconocimiento del contenido de CSV

In [1786]:
# Para mostrar todas las filas y columnas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
retail2_df.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,CustomerName,Email,Address,PhoneNumber,Category,Supplier,StockLevel,Discount,SaleChannel,ReturnStatus,ProductWeight,ProductDimensions,ShippingCost,SalesRegion,PromotionCode,PaymentMethod
0,536578.0,84969,"[""description"": ""BOX OF 6 ASSORTED COLOUR TEAS...",6,12/1/2010 12:28,4.25,17763,United Kingdom,David Johnson,david.johnson@mail.com,450 Ash St,555-734-2925,Beauty,Supplier A,853,17.28,Online,Not Returned,3.81,37x38x83 cm,7.14,North America,SALE15,Bank Transfer
1,536446.0,21756,DOORMAT NEW ENGLAND,100,12/1/2010 10:16,795.0,15939,United Kingdom,Henry Williams,henry.williams@test.org,815 Maple St,555-782-8115,Toys,Supplier A,910,9.08,Online,Not Returned,9.51,8x65x86 cm,12.48,Asia,SALE15,Bank Transfer
2,536633.0,22632,HAND WARMER RED POLKA DOT,6,12/1/2010 13:23,1.85,12295,United Kingdom,Jane Brown,jane.brown@mail.com,253 High St,555-936-2229,Sports,Supplier D,578,45.42,In-Store,Returned,7.35,17x71x89 cm,14.27,North America,DISCOUNT5,Bank Transfer
3,536522.0,22111,"{""description"": ""SCANDINAVIAN REDS RIBBONS""}",10,12/1/2010 11:32,1.65,15685,United Kingdom,Frank Johnson,frank.johnson@example.com,883 Elm St,555-242-3875,Beauty,Supplier C,75,29.17,Online,Returned,6.03,45x4x36 cm,15.54,Asia,NaN,PayPal
4,NaN,22634,"{""description"": ""BAKING SET 9 PIECE RETROSPOT""}",6,12/1/2010 11:07,4.95,11696,United Kingdom,Eva Smith,eva.smith@mail.com,709 Pine St,555-412-6954,Home & Garden,Supplier C,435,20.04,Online,Not Returned,1.64,70x31x19 cm,13.39,Australia,PROMO10,Bank Transfer


## Limpieza de datos por columna 

## Columna Invoice :

- Rellenar los valores Nulos con un valor especifico con 0:Realizado en el DataFrame retail2_df
- Cambiar los datos flotantes a números enteros:Realizado en el DataFrame retail2_df

1. vamos a verificar que tipo de dato la columna InvoiceNo, esta debe es de tipo entero 

In [1787]:
#Verificando que tipo de dato es la columna 
print(retail2_df['InvoiceNo'].dtype)

float64


In [1788]:
# Rellenar los valores NaN con 0 y convertir a int
retail2_df['InvoiceNo'] = retail2_df['InvoiceNo'].fillna(0).astype(int)

# Verifica el tipo de dato
print(retail2_df['InvoiceNo'].dtype)

int64


## Columna StockCode:
- Verificación del tipo del dato  
- Visualización de los datos para realizar los cambios correspondientes.
- Eliminación de espacios en blanco al principio y al final de los valores en la columna
- Movimientos de datos a las columna correspontiendes que no estan relacionados con la columna. 
- Identificacón de los datos inválidos. 
- Relleno de los datos faltantes.

- Verificación tipo de dato

In [1789]:
print(retail2_df['StockCode'].dtype)

object


- Visualización de los datos ordenadamente 

In [1790]:
retail2_df=retail2_df.sort_values(by='StockCode')
print(retail2_df['StockCode'].unique())

['21723' '21724' '21730' '21731' '21754' '21755' '21756' '21777' '21914'
 '22111' '22112' '22139' '22310' '22382' '22384' '22386' '22469' '22570'
 '22571' '22622' '22623' '22624' '22632' '22633' '22634' '22725' '22745'
 '22748' '22749' '22751' '22752' '22754' '22960' '71053' '84029E' '84029G'
 '84406B' '84879' '84907A' '84907B' '84907C' '84969' '84988' '85099B'
 '85123A' 'BOX OF 6 ASSORTED COLOUR TEASPOONS' 'HAND WARMER RED POLKA DOT'
 'JAM MAKING SET WITH JARS' 'LOVE BUILDING BLOCK WORD'
 'RED  HARMONICA IN BOX' 'RED HARMONICA IN BOX'
 'SCANDINAVIAN REDS RIBBONS' 'SET 2 TEA TOWELS I LOVE LONDON'
 'STRIPED CHARLIE+LOLA CHARLOTTE BAG'
 '{"description": ["RED WOOLLY HOTTIE WHITE HEART.: details"]}']


- Eliminación de espacios en blanco al principio y al final de los valores en la columna

In [1791]:
retail2_df['StockCode'] = retail2_df['StockCode'].str.strip()

# Mostrar el DataFrame para verificar los cambios
print(retail2_df['StockCode'].head())

291    21723
34     21723
165    21723
40     21723
151    21723
Name: StockCode, dtype: object


- Movimientos de datos a las columna correspontiendes que no estan relacionados con la columna.
En este caso se encotro datos de la columna Description por lo tanto se transladan a ella.  

In [1792]:
# Mover los datos de StockCode a Description si StockCode contiene texto y no comienza con un número
mask = retail2_df['StockCode'].apply(lambda x: isinstance(x, str) and not x[0].isdigit())
retail2_df.loc[mask, 'Description'] = retail2_df.loc[mask, 'StockCode']

# Limpiar la columna StockCode para eliminar los datos que se movieron a Description
retail2_df.loc[mask, 'StockCode'] = ''

# Ordenar la salida por el índice
retail2_df_sorted = retail2_df.sort_index()

# Mostrar las columnas StockCode y Description
print(retail2_df_sorted[['StockCode', 'Description']])

    StockCode                                        Description
0       84969  ["description": "BOX OF 6 ASSORTED COLOUR TEAS...
1       21756                                DOORMAT NEW ENGLAND
2       22632                          HAND WARMER RED POLKA DOT
3       22111       {"description": "SCANDINAVIAN REDS RIBBONS"}
4       22634    {"description": "BAKING SET 9 PIECE RETROSPOT"}
5       22960      {"description": ["JAM MAKING SET WITH JARS"]}
6       22139                    RETROSPOT TEA SET CERAMIC 11 PC
7       22112                         CHOCOLATE HOT WATER BOTTLE
8       22634                       BAKING SET 9 PIECE RETROSPOT
9       22749                                                NaN
10      22632                          HAND WARMER RED POLKA DOT
11      22112                         CHOCOLATE HOT WATER BOTTLE
12      21724                               RED HARMONICA IN BOX
13      71053                                WHITE METAL LANTERN
14     84029G            

- Confirmación de los datos luego del traslado realizado.

In [1793]:
retail2_df=retail2_df.sort_values(by='StockCode')
print(retail2_df['StockCode'].unique())

['' '21723' '21724' '21730' '21731' '21754' '21755' '21756' '21777'
 '21914' '22111' '22112' '22139' '22310' '22382' '22384' '22386' '22469'
 '22570' '22571' '22622' '22623' '22624' '22632' '22633' '22634' '22725'
 '22745' '22748' '22749' '22751' '22752' '22754' '22960' '71053' '84029E'
 '84029G' '84406B' '84879' '84907A' '84907B' '84907C' '84969' '84988'
 '85099B' '85123A']


- Identificación de los datos inválidos.

In [1794]:
# Identificar los datos inválidos en la columna StockCode, incluyendo NaN y vacíos
invalid_data_stockcode = retail2_df[retail2_df['StockCode'].isnull() | (retail2_df['StockCode'] == '') | (retail2_df['StockCode'].str.lower() == 'nan')]

# Mostrar los datos inválidos y la cantidad
print(invalid_data_stockcode[['StockCode','Description']])
print(f"Cantidad de datos inválidos: {invalid_data_stockcode.shape[0]}")

    StockCode                                        Description
314            {"description": ["RED WOOLLY HOTTIE WHITE HEAR...
49                                SET 2 TEA TOWELS I LOVE LONDON
322                                    SCANDINAVIAN REDS RIBBONS
316                                         RED HARMONICA IN BOX
298                                        RED  HARMONICA IN BOX
292                                     LOVE BUILDING BLOCK WORD
369                                     JAM MAKING SET WITH JARS
272                                    HAND WARMER RED POLKA DOT
136                           BOX OF 6 ASSORTED COLOUR TEASPOONS
213                           STRIPED CHARLIE+LOLA CHARLOTTE BAG
Cantidad de datos inválidos: 10


Los datos inválidos corresponden a campos vacíos sin información.
Por lo anterior se rellena los valores faltantes en la columna ‘StockCode’ utilizando un diccionario de referencia basado en la columna ‘Description’  y luego identificaremos y tendremos los datos que aún son inválidos.

In [1795]:
#Crear un diccionario de referencia
reference_dict_stockcode = retail2_df.dropna(subset=['StockCode']).set_index('Description')['StockCode'].to_dict()

# Función para rellenar los valores faltantes
def fill_missing_descriptions(row):
    if pd.isna(row['StockCode']) or row['StockCode'] == '':
        return reference_dict_stockcode.get(row['Description'], row['StockCode'])
    return row['StockCode']

# Aplicar la función al DataFrame
retail2_df['StockCode'] = retail2_df.apply(fill_missing_descriptions, axis=1)

# Identificar datos inválidos
invalid_data_stockcode = retail2_df[retail2_df['StockCode'].isnull() | (retail2_df['StockCode'] == '') | (retail2_df['StockCode'].str.lower() == 'nan')]

# Mostrar los datos inválidos y la cantidad
print(invalid_data_stockcode[['StockCode', 'Description']])
print(f"Cantidad de datos inválidos: {invalid_data_stockcode.shape[0]}")

    StockCode                                        Description
314            {"description": ["RED WOOLLY HOTTIE WHITE HEAR...
Cantidad de datos inválidos: 1


## Explicación del código

- 1. Crear un diccionario de referencia:
retail2_df.dropna(subset=['StockCode']): Elimina las filas donde StockCode es NaN.
.set_index('Description')['StockCode']: Establece Description como índice y selecciona la columna StockCode.
.to_dict(): Convierte el resultado en un diccionario donde las claves son las descripciones y los valores son los códigos de stock.

- 2. Definir una función para rellenar los valores faltantes:
if pd.isna(row['StockCode']) or row['StockCode'] == '': Verifica si StockCode es NaN o está vacío.
return reference_dict_stockcode.get(row['Description'], row['StockCode']): Busca en el diccionario reference_dict_stockcode el StockCode correspondiente a la Description. Si no encuentra una coincidencia, devuelve el valor original de StockCode.
return row['StockCode']: Si StockCode no es NaN ni está vacío, devuelve el valor original.

- 3. Aplicar la función al DataFrame:
retail2_df.apply(fill_missing_descriptions, axis=1): Aplica la función fill_missing_descriptions a cada fila del DataFrame retail2_df. axis=1 indica que la función se aplica a lo largo de las filas.

- 4. retail2_df['StockCode'].isnull(): Verifica si StockCode es NaN.
(retail2_df['StockCode'] == ''): Verifica si StockCode está vacío.
(retail2_df['StockCode'].str.lower() == 'nan'): Verifica si StockCode contiene la cadena 'nan' (en minúsculas).
retail2_df[...]: Filtra el DataFrame para incluir solo las filas que cumplen alguna de estas condiciones.


Se observa que aun queda un dato inválido que al momento de corregir los datos correspondientes a la columna Description se solucionara.

## Columna Description 

- Verificación del tipo del dato  
- Visualización de los datos para realizar los cambios correspondientes.
- Eliminación de espacios en blanco al principio y al final de los valores en la columna
- Función para limpiar y unificar las descripciones :Eliminación de palabras y caracteres no correspondientes a la descripcion.
- Se realiza los cambios minimos con las descripciones especificas a las cuales la función no reconocio. 
- Identificar los datos inválidos en la columna Description, incluyendo NaN y vacíos
- Rellenar los valores faltantes en la columna ‘Description’ utilizando un diccionario de referencia basado en la columna ‘StockCode’.

- Verificación del tipo del dato 

In [1796]:
print(retail2_df['Description'].dtype)

object


- Visualización de los datos para realizar los cambios correspondientes.

In [1797]:
# Ordenar el DataFrame por la columna 'Description'
retail2_df = retail2_df.sort_values(by='Description')

# Obtener los valores únicos y contarlos
unique_descriptions = retail2_df['Description'].unique()
unique_count = retail2_df['Description'].nunique()

# Mostrar los valores únicos y la cantidad
print(unique_descriptions)
print(f"Cantidad de valores únicos: {unique_count}")

['  BAKING SET 9 PIECE RETROSPOT  ' '  BOX OF VINTAGE ALPHABET BLOCKS  '
 '  BOX OF VINTAGE JIGSAW BLOCKS  ' '  CHOCOLATE HOT WATER BOTTLE  '
 '  CREAM CUPID HEARTS COAT HANGER  ' '  HAND WARMER RED POLKA DOT  '
 '  HAND WARMER UNION JACK  ' '  HOME BUILDING BLOCK WORD  '
 '  JUMBO BAG RED RETROSPOT  ' '  PACK OF 12 BLUE RETROSPOT TISSUES  '
 '  PACK OF 6 SMALL FRUIT STRAWS  ' "  POPPY'S PLAYHOUSE BEDROOM  "
 "  POPPY'S PLAYHOUSE KITCHEN  " '  RED HARMONICA IN BOX  '
 '  RED TOADSTOOL LED NIGHT LIGHT  ' '  RETROSPOT TEA SET CERAMIC 11 PC  '
 '  WHITE METAL LANTERN  ' '  nan  ' '***BAKING SET 9 PIECE RETROSPOT***'
 '***CHOCOLATE HOT WATER BOTTLE***' '***DOORMAT NEW ENGLAND***'
 '***FELTCRAFT PRINCESS CHARLOTTE DOLL***'
 '***HAND WARMER RED POLKA DOT***' '***IVORY KNITTED MUG COSY***'
 '***JAM MAKING SET WITH JARS***' "***POPPY'S PLAYHOUSE BEDROOM***"
 "***POPPY'S PLAYHOUSE KITCHEN***" '***RED GLASS CANDLE HOLDER STAR***'
 '***RED TOADSTOOL LED NIGHT LIGHT***'
 '***RED WOOLLY HOTTIE WHIT

- Eliminación de espacios en blanco al principio y al final de los valores en la columna

In [1798]:
retail2_df['Description'] = retail2_df['Description'].str.strip()

# Mostrar el DataFrame para verificar los cambios
print(retail2_df['Description'].head())

70       BAKING SET 9 PIECE RETROSPOT
29     BOX OF VINTAGE ALPHABET BLOCKS
388      BOX OF VINTAGE JIGSAW BLOCKS
102        CHOCOLATE HOT WATER BOTTLE
198    CREAM CUPID HEARTS COAT HANGER
Name: Description, dtype: object


- Función para limpiar y unificar las descripciones según su mayoria de coincidencias y la eliminación de datos no correspondientes 

In [1799]:
# Función para limpiar y unificar las descripciones
def clean_description(desc):
    # Eliminar las palabras "description" y "details" si están presentes
    desc = re.sub(r'description|details', '', desc, flags=re.IGNORECASE)
    # Eliminar caracteres especiales y convertir a mayúsculas
    desc = re.sub(r'[^a-zA-Z0-9\s]', '', desc).upper()
    # Eliminar espacios adicionales
    desc = re.sub(r'\s+', ' ', desc).strip()
    return desc

# Aplicar la función de limpieza a la columna 'Description'
retail2_df['Description'] = retail2_df['Description'].apply(lambda x: clean_description(x) if isinstance(x, str) else x)

# Reemplazar "INVALID123", 0 y NaN con cadenas vacías
retail2_df['Description'] = retail2_df['Description'].replace(['INVALID123', 0, np.nan], '')

# Ordenar el DataFrame por la columna 'Description'
retail2_df = retail2_df.sort_values(by='Description')

# Imprimir los valores únicos y la cantidad de valores únicos
print(retail2_df['Description'].unique())
print(f"Cantidad de valores únicos: {retail2_df['Description'].nunique()}")



['' 'ASSORTED COLOUR BIRD ORNAMENT' 'BAKING SET 9 PIECE RETROSPOT'
 'BLUE HARMONICA IN BOX' 'BOX OF 6 ASSORTED COLOUR TEASPOONS'
 'BOX OF VINTAGE ALPHABET BLOCKS' 'BOX OF VINTAGE JIGSAW BLOCKS'
 'CHOCOLATE HOT WATER BOTTLE' 'COFFEE MUG APPLES DESIGN'
 'CREAM CUPID HEARTS COAT HANGER' 'DOORMAT NEW ENGLAND'
 'FELTCRAFT PRINCESS CHARLOTTE DOLL' 'GLASS JAR DAISY FRESH COTTON WOOL'
 'GLASS STAR FROSTED TLIGHT HOLDER' 'HAND WARMER RED POLKA DOT'
 'HAND WARMER UNION JACK' 'HANGING STAR WITH BELLS'
 'HEART OF WICKER LARGE' 'HOME BUILDING BLOCK WORD'
 'IVORY KNITTED MUG COSY' 'JAM MAKING SET WITH JARS'
 'JUMBO BAG PINK POLKADOT' 'JUMBO BAG RED RETROSPOT'
 'KNITTED UNION FLAG HOT WATER BOTTLE' 'LOVE BUILDING BLOCK WORD' 'NAN'
 'PACK OF 12 BLUE RETROSPOT TISSUES' 'PACK OF 12 PINK POLKADOT TISSUES'
 'PACK OF 12 RED RETROSPOT TISSUES' 'PACK OF 6 SMALL FRUIT STRAWS'
 'POPPYS PLAYHOUSE BEDROOM' 'POPPYS PLAYHOUSE KITCHEN'
 'RECIPE BOX WITH METAL HEART' 'RED GLASS CANDLE HOLDER STAR'
 'RED HARMONICA IN

- Se realiza los cambios minimos con las descripciones no válidas. 

In [1800]:
# Usar loc para identificar y reemplazar el valor específico
retail2_df.loc[retail2_df['Description'] == 'SET OF 7 BABUSHKA NESTING BOXES', 'Description'] = 'SET 7 BABUSHKA NESTING BOXES'

# Verificar el cambio
print(retail2_df['Description'].unique())
print(f"Cantidad de valores únicos: {retail2_df['Description'].nunique()}")



['' 'ASSORTED COLOUR BIRD ORNAMENT' 'BAKING SET 9 PIECE RETROSPOT'
 'BLUE HARMONICA IN BOX' 'BOX OF 6 ASSORTED COLOUR TEASPOONS'
 'BOX OF VINTAGE ALPHABET BLOCKS' 'BOX OF VINTAGE JIGSAW BLOCKS'
 'CHOCOLATE HOT WATER BOTTLE' 'COFFEE MUG APPLES DESIGN'
 'CREAM CUPID HEARTS COAT HANGER' 'DOORMAT NEW ENGLAND'
 'FELTCRAFT PRINCESS CHARLOTTE DOLL' 'GLASS JAR DAISY FRESH COTTON WOOL'
 'GLASS STAR FROSTED TLIGHT HOLDER' 'HAND WARMER RED POLKA DOT'
 'HAND WARMER UNION JACK' 'HANGING STAR WITH BELLS'
 'HEART OF WICKER LARGE' 'HOME BUILDING BLOCK WORD'
 'IVORY KNITTED MUG COSY' 'JAM MAKING SET WITH JARS'
 'JUMBO BAG PINK POLKADOT' 'JUMBO BAG RED RETROSPOT'
 'KNITTED UNION FLAG HOT WATER BOTTLE' 'LOVE BUILDING BLOCK WORD' 'NAN'
 'PACK OF 12 BLUE RETROSPOT TISSUES' 'PACK OF 12 PINK POLKADOT TISSUES'
 'PACK OF 12 RED RETROSPOT TISSUES' 'PACK OF 6 SMALL FRUIT STRAWS'
 'POPPYS PLAYHOUSE BEDROOM' 'POPPYS PLAYHOUSE KITCHEN'
 'RECIPE BOX WITH METAL HEART' 'RED GLASS CANDLE HOLDER STAR'
 'RED HARMONICA IN

In [1801]:
retail2_df['Description'] = retail2_df['Description'].replace({
    'STRIPED CHARLIELOLA CHARLOTTE BAG': 'STRIPED CHARLIE+LOLA CHARLOTTE BAG',
    'WHITE HANGING HEART TLIGHT HOLDER': 'WHITE HANGING HEART T-LIGHT HOLDER'
})

# Verificar los cambios
print(retail2_df['Description'].unique())

['' 'ASSORTED COLOUR BIRD ORNAMENT' 'BAKING SET 9 PIECE RETROSPOT'
 'BLUE HARMONICA IN BOX' 'BOX OF 6 ASSORTED COLOUR TEASPOONS'
 'BOX OF VINTAGE ALPHABET BLOCKS' 'BOX OF VINTAGE JIGSAW BLOCKS'
 'CHOCOLATE HOT WATER BOTTLE' 'COFFEE MUG APPLES DESIGN'
 'CREAM CUPID HEARTS COAT HANGER' 'DOORMAT NEW ENGLAND'
 'FELTCRAFT PRINCESS CHARLOTTE DOLL' 'GLASS JAR DAISY FRESH COTTON WOOL'
 'GLASS STAR FROSTED TLIGHT HOLDER' 'HAND WARMER RED POLKA DOT'
 'HAND WARMER UNION JACK' 'HANGING STAR WITH BELLS'
 'HEART OF WICKER LARGE' 'HOME BUILDING BLOCK WORD'
 'IVORY KNITTED MUG COSY' 'JAM MAKING SET WITH JARS'
 'JUMBO BAG PINK POLKADOT' 'JUMBO BAG RED RETROSPOT'
 'KNITTED UNION FLAG HOT WATER BOTTLE' 'LOVE BUILDING BLOCK WORD' 'NAN'
 'PACK OF 12 BLUE RETROSPOT TISSUES' 'PACK OF 12 PINK POLKADOT TISSUES'
 'PACK OF 12 RED RETROSPOT TISSUES' 'PACK OF 6 SMALL FRUIT STRAWS'
 'POPPYS PLAYHOUSE BEDROOM' 'POPPYS PLAYHOUSE KITCHEN'
 'RECIPE BOX WITH METAL HEART' 'RED GLASS CANDLE HOLDER STAR'
 'RED HARMONICA IN

 - Identificación de los datos inválidos en la columna Description, incluyendo NaN y vacíos

In [1802]:
# Identificar los datos inválidos en la columna Description, incluyendo NaN y vacíos
invalid_data = retail2_df[retail2_df['Description'].isnull() | (retail2_df['Description'] == '') | (retail2_df['Description'].str.lower() == 'nan')]

# Mostrar los datos inválidos y la cantidad
print(invalid_data[['StockCode','Description']])
print(f"Cantidad de datos inválidos: {invalid_data.shape[0]}")

    StockCode Description
208    85123A            
259    85123A            
252     22623            
251     22632            
37      22622            
186     22382            
168     22634            
315     22139            
274     21755            
390     21723            
360     84879            
227     21724            
203     21724            
345     21724            
239     21755            
309     21756            
145     22960            
223     21730            
9       22749            
182     22745            
43      22634            
319     22634            
276     22633            
130     22632            
169     22752            
401     22386            
115     22310            
32      21914            
243     22632            
348     21914         NAN
Cantidad de datos inválidos: 30


- Rellenar los valores faltantes en la columna ‘Description’ utilizando un diccionario de referencia basado en la columna ‘StockCode’.

In [1803]:
# Crear un diccionario de referencia
reference_dict = retail2_df.dropna(subset=['Description']).set_index('StockCode')['Description'].to_dict()

# Función para rellenar los valores faltantes
def fill_missing_description(row):
    if pd.isna(row['Description']) or row['Description'] in ['', '0']:
        return reference_dict.get(row['StockCode'], row['Description'])
    return row['Description']

# Aplicar la función al DataFrame
retail2_df['Description'] = retail2_df.apply(fill_missing_description, axis=1)

# Ordenar el DataFrame por la columna 'Description'
retail2_df = retail2_df.sort_values(by='Description')

# Imprimir los valores únicos y la cantidad de valores únicos
print(retail2_df['Description'].unique())
print(f"Cantidad de valores únicos: {retail2_df['Description'].nunique()}")

['' 'ASSORTED COLOUR BIRD ORNAMENT' 'BAKING SET 9 PIECE RETROSPOT'
 'BLUE HARMONICA IN BOX' 'BOX OF 6 ASSORTED COLOUR TEASPOONS'
 'BOX OF VINTAGE ALPHABET BLOCKS' 'BOX OF VINTAGE JIGSAW BLOCKS'
 'CHOCOLATE HOT WATER BOTTLE' 'COFFEE MUG APPLES DESIGN'
 'CREAM CUPID HEARTS COAT HANGER' 'DOORMAT NEW ENGLAND'
 'FELTCRAFT PRINCESS CHARLOTTE DOLL' 'GLASS JAR DAISY FRESH COTTON WOOL'
 'GLASS STAR FROSTED TLIGHT HOLDER' 'HAND WARMER RED POLKA DOT'
 'HAND WARMER UNION JACK' 'HANGING STAR WITH BELLS'
 'HEART OF WICKER LARGE' 'HOME BUILDING BLOCK WORD'
 'IVORY KNITTED MUG COSY' 'JAM MAKING SET WITH JARS'
 'JUMBO BAG PINK POLKADOT' 'JUMBO BAG RED RETROSPOT'
 'KNITTED UNION FLAG HOT WATER BOTTLE' 'LOVE BUILDING BLOCK WORD' 'NAN'
 'PACK OF 12 BLUE RETROSPOT TISSUES' 'PACK OF 12 PINK POLKADOT TISSUES'
 'PACK OF 12 RED RETROSPOT TISSUES' 'PACK OF 6 SMALL FRUIT STRAWS'
 'POPPYS PLAYHOUSE BEDROOM' 'POPPYS PLAYHOUSE KITCHEN'
 'RECIPE BOX WITH METAL HEART' 'RED GLASS CANDLE HOLDER STAR'
 'RED HARMONICA IN

In [1804]:
print(retail2_df[['InvoiceNo','StockCode','Description']])

     InvoiceNo StockCode                          Description
186     536407     22382                                     
352     536473     84879        ASSORTED COLOUR BIRD ORNAMENT
96      536734     84879        ASSORTED COLOUR BIRD ORNAMENT
257     536573     84879        ASSORTED COLOUR BIRD ORNAMENT
318     536673     84879        ASSORTED COLOUR BIRD ORNAMENT
80      536634     84879        ASSORTED COLOUR BIRD ORNAMENT
189     536420     84879        ASSORTED COLOUR BIRD ORNAMENT
360     536367     84879        ASSORTED COLOUR BIRD ORNAMENT
28      536534     84879        ASSORTED COLOUR BIRD ORNAMENT
296     536697     22634         BAKING SET 9 PIECE RETROSPOT
425     536660     22634         BAKING SET 9 PIECE RETROSPOT
279     536709     22634         BAKING SET 9 PIECE RETROSPOT
380     536444     22634         BAKING SET 9 PIECE RETROSPOT
167          0     22634         BAKING SET 9 PIECE RETROSPOT
39      536448     22634         BAKING SET 9 PIECE RETROSPOT
8       

In [1805]:

print(retail2_df.loc[251, ['InvoiceNo', 'StockCode','Description', 'InvoiceDate']])

InvoiceNo                              0
StockCode                          22632
Description    HAND WARMER RED POLKA DOT
InvoiceDate              12/1/2010 12:53
Name: 251, dtype: object


- Posterior a lo anterior se identifica nuevamente los datos inválidos

In [1806]:
# Identificar los datos inválidos en la columna Description, incluyendo NaN y vacíos
invalid_data = retail2_df[retail2_df['Description'].isnull() | (retail2_df['Description'] == '') | (retail2_df['Description'].str.lower() == 'nan')]

# Mostrar los datos inválidos y la cantidad
print(invalid_data[['StockCode','Description']])
print(f"Cantidad de datos inválidos: {invalid_data.shape[0]}")

    StockCode Description
186     22382            
348     21914         NAN
32      21914         NAN
Cantidad de datos inválidos: 3


In [1807]:
# Actualizar el código 22382 a 22386 y la descripción correspondiente
retail2_df.loc[retail2_df['StockCode'] == 22382, 'StockCode'] = 22386
retail2_df.loc[retail2_df['StockCode'] == 22386, 'Description'] = 'JUMBO BAG PINK POLKADOT'

# Actualizar la descripción para los índices 348 y 32 con el código 21914
indices_a_actualizar = [348, 32]
retail2_df.loc[indices_a_actualizar, 'Description'] = 'BLUE HARMONICA IN BOX'



In [1808]:
# Actualizar el código 22382 a 22386 en el índice 186
retail2_df.loc[186, 'StockCode'] = 22386
retail2_df.loc[186, 'Description'] = 'JUMBO BAG PINK POLKADOT'

In [1809]:
# Identificar los datos inválidos en la columna Description, incluyendo NaN y vacíos
invalid_data = retail2_df[retail2_df['Description'].isnull() | (retail2_df['Description'] == '') | (retail2_df['Description'].str.lower() == 'nan')]

# Mostrar los datos inválidos y la cantidad
print(invalid_data[['StockCode','Description']])
print(f"Cantidad de datos inválidos: {invalid_data.shape[0]}")

Empty DataFrame
Columns: [StockCode, Description]
Index: []
Cantidad de datos inválidos: 0


## Columna InvoiceDate

- Verificación de datos
- Eliminación de texto no correspondientes al formato de fechas : palabra 'Date'.
- Convertir la columna InvoiceDate a datetime con el formato correcto que es dia, mes, año y hora. 
- Defenición de fechas: En este caso fueron las ventas correspondiente a un solo dia  12/01/2010 y con su respectiva hora
- Verificación de datos inválidos:Encontramos fechas correspondiente a años como 2050, 1900, dias no correspondientes a la fecha a trabajar 30/02/2010, formatos de fechas no válidas como mes, dia, año y caracteres no correspondientes como -, se obtiene columnas vacías. 
- Correción de los datos inválidos mencionados anteriormente.
- Limpiar y corregir los números de factura y las fechas en la columna ‘InvoiceDate’, asegurando que todas las facturas tengan fechas válidas y consistentes: Para los datos sin fecha se tiene encuanta el consecutivo de la hora tomando de refencia la columna de las facturas existentes en la columna InvoiceDate.
- Corrección de los valores de ‘InvoiceNo’ y ‘InvoiceDate’ para ciertas filas específicas, asegurando que las fechas y números de factura sean correctos y estén en el formato adecuado. 

- Verificación de datos 

In [1810]:
# Verificación de datos y se observan en orden 
retail2_df = retail2_df.sort_values(by='InvoiceDate') 
print(retail2_df['InvoiceDate'].unique())

['01-12-2010 09:47' '01-12-2010 10:09' '01-12-2010 11:27'
 '01-12-2010 13:42' '01/01/1900 00:00' '01/01/2050 00:00'
 '01/12/2010 25:61' '12/01/2010 09:08' '12/01/2010 10:41'
 '12/01/2010 10:52' '12/01/2010 12:22' '12/01/2010 14:55'
 '12/1/2010 10:00' '12/1/2010 10:01' '12/1/2010 10:03' '12/1/2010 10:04'
 '12/1/2010 10:05' '12/1/2010 10:06' '12/1/2010 10:07' '12/1/2010 10:08'
 '12/1/2010 10:10' '12/1/2010 10:12' '12/1/2010 10:13' '12/1/2010 10:15'
 '12/1/2010 10:16' '12/1/2010 10:17' '12/1/2010 10:18' '12/1/2010 10:20'
 '12/1/2010 10:21' '12/1/2010 10:22' '12/1/2010 10:23' '12/1/2010 10:24'
 '12/1/2010 10:26' '12/1/2010 10:27' '12/1/2010 10:28' '12/1/2010 10:29'
 '12/1/2010 10:30' '12/1/2010 10:31' '12/1/2010 10:32' '12/1/2010 10:33'
 '12/1/2010 10:35' '12/1/2010 10:36' '12/1/2010 10:37' '12/1/2010 10:38'
 '12/1/2010 10:39' '12/1/2010 10:40' '12/1/2010 10:42' '12/1/2010 10:44'
 '12/1/2010 10:46' '12/1/2010 10:47' '12/1/2010 10:48' '12/1/2010 10:49'
 '12/1/2010 10:50' '12/1/2010 10:51' '

- Eliminación de texto no correspondientes al formato de fechas : palabra 'Date'.

In [1811]:
# Eliminar las palabras 'Date:'
retail2_df['InvoiceDate'] = retail2_df['InvoiceDate'].str.replace(r'Date:', '', regex=True).str.strip()
retail2_df = retail2_df.sort_values(by='InvoiceDate')
print(retail2_df['InvoiceDate'].unique())

['01-12-2010 09:47' '01-12-2010 10:09' '01-12-2010 11:27'
 '01-12-2010 13:42' '01/01/1900 00:00' '01/01/2050 00:00'
 '01/12/2010 25:61' '12/01/2010 09:08' '12/01/2010 10:41'
 '12/01/2010 10:52' '12/01/2010 12:22' '12/01/2010 14:55'
 '12/1/2010 10:00' '12/1/2010 10:01' '12/1/2010 10:03' '12/1/2010 10:04'
 '12/1/2010 10:05' '12/1/2010 10:06' '12/1/2010 10:07' '12/1/2010 10:08'
 '12/1/2010 10:10' '12/1/2010 10:12' '12/1/2010 10:13' '12/1/2010 10:15'
 '12/1/2010 10:16' '12/1/2010 10:17' '12/1/2010 10:18' '12/1/2010 10:20'
 '12/1/2010 10:21' '12/1/2010 10:22' '12/1/2010 10:23' '12/1/2010 10:24'
 '12/1/2010 10:26' '12/1/2010 10:27' '12/1/2010 10:28' '12/1/2010 10:29'
 '12/1/2010 10:30' '12/1/2010 10:31' '12/1/2010 10:32' '12/1/2010 10:33'
 '12/1/2010 10:35' '12/1/2010 10:36' '12/1/2010 10:37' '12/1/2010 10:38'
 '12/1/2010 10:39' '12/1/2010 10:40' '12/1/2010 10:42' '12/1/2010 10:44'
 '12/1/2010 10:46' '12/1/2010 10:47' '12/1/2010 10:48' '12/1/2010 10:49'
 '12/1/2010 10:50' '12/1/2010 10:51' '

- Convertir la columna InvoiceDate a datetime con el formato correcto que es dia, mes, año y hora. En este caso fueron las ventas correspondiente a un solo dia  12/01/2010 y con su respectiva hora

In [1812]:
# Convertir la columna InvoiceDate a datetime con el formato correcto
retail2_df['InvoiceDate'] = pd.to_datetime(retail2_df['InvoiceDate'], format='%d/%m/%Y %H:%M', errors='coerce')

# Ordenar el DataFrame por la columna InvoiceDate
retail2_df = retail2_df.sort_values(by='InvoiceDate')

# Convertir la columna InvoiceDate de nuevo a string en el formato deseado
retail2_df['InvoiceDate'] = retail2_df['InvoiceDate'].dt.strftime('%d/%m/%Y %H:%M')

# Mostrar los valores únicos de la columna InvoiceDate
print(retail2_df['InvoiceDate'].unique())

['01/01/1900 00:00' '12/01/2010 08:26' '12/01/2010 08:28'
 '12/01/2010 08:34' '12/01/2010 08:35' '12/01/2010 09:00'
 '12/01/2010 09:01' '12/01/2010 09:02' '12/01/2010 09:03'
 '12/01/2010 09:06' '12/01/2010 09:07' '12/01/2010 09:08'
 '12/01/2010 09:09' '12/01/2010 09:11' '12/01/2010 09:12'
 '12/01/2010 09:13' '12/01/2010 09:14' '12/01/2010 09:15'
 '12/01/2010 09:16' '12/01/2010 09:17' '12/01/2010 09:19'
 '12/01/2010 09:21' '12/01/2010 09:22' '12/01/2010 09:23'
 '12/01/2010 09:24' '12/01/2010 09:25' '12/01/2010 09:26'
 '12/01/2010 09:27' '12/01/2010 09:28' '12/01/2010 09:29'
 '12/01/2010 09:30' '12/01/2010 09:31' '12/01/2010 09:32'
 '12/01/2010 09:33' '12/01/2010 09:34' '12/01/2010 09:35'
 '12/01/2010 09:36' '12/01/2010 09:38' '12/01/2010 09:39'
 '12/01/2010 09:40' '12/01/2010 09:41' '12/01/2010 09:42'
 '12/01/2010 09:43' '12/01/2010 09:44' '12/01/2010 09:45'
 '12/01/2010 09:46' '12/01/2010 09:48' '12/01/2010 09:49'
 '12/01/2010 09:50' '12/01/2010 09:51' '12/01/2010 09:52'
 '12/01/2010 0

- Defenición de fechas: En este caso fueron las ventas correspondiente a un solo dia  12/01/2010 y con su respectiva hora
- Verificación de datos inválidos

In [1813]:
# Intentar convertir la columna InvoiceDate a tipo datetime con el formato correcto
retail2_df['InvoiceDate'] = pd.to_datetime(retail2_df['InvoiceDate'], format='%d/%m/%Y %H:%M', errors='coerce')

# Definir la fecha válida
valid_date = pd.to_datetime('12/01/2010', format='%d/%m/%Y')

# Filtrar las fechas que no coinciden con la fecha válida (manteniendo la hora)
invalid_dates = retail2_df[retail2_df['InvoiceDate'].dt.date != valid_date.date()]

# Mostrar únicamente las columnas InvoiceNo y InvoiceDate de las filas con fechas inválidas
print(invalid_dates[['InvoiceNo', 'InvoiceDate']])

# Convertir la columna InvoiceDate de nuevo a string en el formato deseado
retail2_df['InvoiceDate'] = retail2_df['InvoiceDate'].dt.strftime('%d/%m/%Y %H:%M')

# Contar las filas con fechas inválidas
invalid_count = invalid_dates.shape[0]
print(f"Cantidad de filas con fechas inválidas: {invalid_count}")

     InvoiceNo InvoiceDate
87      536677  1900-01-01
338     536380  1900-01-01
352     536473  1900-01-01
83      536598  1900-01-01
268     536636  1900-01-01
139     536687  1900-01-01
100     536588  1900-01-01
358     536425  1900-01-01
186     536407  1900-01-01
160     536544  1900-01-01
6       536707  2050-01-01
142     536374  2050-01-01
78           0  2050-01-01
308     536449  2050-01-01
104     536564  2050-01-01
173     536519  2050-01-01
241     536530  2050-01-01
380     536444  2050-01-01
279     536709  2050-01-01
34      536651  2050-01-01
69      536564  2050-01-01
223     536417         NaT
32      536439         NaT
114     536517         NaT
396     536652         NaT
412          0         NaT
52           0         NaT
174     536390         NaT
285     536669         NaT
318     536673         NaT
281     536630         NaT
90      536605         NaT
430     536370         NaT
58      536455         NaT
82      536432         NaT
105     536630         NaT
1

En el los datos inválidos encontramos fechas correspondiente a años como 2050, 1900, dias no correspondientes a la fecha a trabajar 30/02/2010, formatos de fechas no válidas como mes, dia, año y caracteres no correspondientes como -

## Correción de los datos invalidos mencionados anteriormente

- Cambio el formato de las fechas inválidas como caracteres

In [1814]:
# Asegurarse de que la columna InvoiceDate esté en formato string
retail2_df['InvoiceDate'] = retail2_df['InvoiceDate'].astype(str)

# Cambiar el formato de las fechas en la columna InvoiceDate
retail2_df['InvoiceDate'] = retail2_df['InvoiceDate'].str.replace('-', '/')
retail2_df['InvoiceDate'] = retail2_df['InvoiceDate'].str.replace('01/12', '12/01')



# Ordenar el DataFrame por la columna InvoiceDate
retail2_df = retail2_df.sort_values(by='InvoiceDate')

# Mostrar los valores únicos de la columna InvoiceDate
print(retail2_df['InvoiceDate'].unique())

['01/01/1900 00:00' '01/01/2050 00:00' '12/01/2010 08:26'
 '12/01/2010 08:28' '12/01/2010 08:34' '12/01/2010 08:35'
 '12/01/2010 09:00' '12/01/2010 09:01' '12/01/2010 09:02'
 '12/01/2010 09:03' '12/01/2010 09:06' '12/01/2010 09:07'
 '12/01/2010 09:08' '12/01/2010 09:09' '12/01/2010 09:11'
 '12/01/2010 09:12' '12/01/2010 09:13' '12/01/2010 09:14'
 '12/01/2010 09:15' '12/01/2010 09:16' '12/01/2010 09:17'
 '12/01/2010 09:19' '12/01/2010 09:21' '12/01/2010 09:22'
 '12/01/2010 09:23' '12/01/2010 09:24' '12/01/2010 09:25'
 '12/01/2010 09:26' '12/01/2010 09:27' '12/01/2010 09:28'
 '12/01/2010 09:29' '12/01/2010 09:30' '12/01/2010 09:31'
 '12/01/2010 09:32' '12/01/2010 09:33' '12/01/2010 09:34'
 '12/01/2010 09:35' '12/01/2010 09:36' '12/01/2010 09:38'
 '12/01/2010 09:39' '12/01/2010 09:40' '12/01/2010 09:41'
 '12/01/2010 09:42' '12/01/2010 09:43' '12/01/2010 09:44'
 '12/01/2010 09:45' '12/01/2010 09:46' '12/01/2010 09:48'
 '12/01/2010 09:49' '12/01/2010 09:50' '12/01/2010 09:51'
 '12/01/2010 0

In [1815]:
print(retail2_df[['InvoiceNo','InvoiceDate']])

     InvoiceNo       InvoiceDate
87      536677  01/01/1900 00:00
186     536407  01/01/1900 00:00
358     536425  01/01/1900 00:00
100     536588  01/01/1900 00:00
139     536687  01/01/1900 00:00
160     536544  01/01/1900 00:00
83      536598  01/01/1900 00:00
352     536473  01/01/1900 00:00
338     536380  01/01/1900 00:00
268     536636  01/01/1900 00:00
69      536564  01/01/2050 00:00
78           0  01/01/2050 00:00
308     536449  01/01/2050 00:00
104     536564  01/01/2050 00:00
173     536519  01/01/2050 00:00
241     536530  01/01/2050 00:00
380     536444  01/01/2050 00:00
279     536709  01/01/2050 00:00
34      536651  01/01/2050 00:00
142     536374  01/01/2050 00:00
6       536707  01/01/2050 00:00
307     536365  12/01/2010 08:26
313     536365  12/01/2010 08:26
175     536365  12/01/2010 08:26
38      536365  12/01/2010 08:26
47      536365  12/01/2010 08:26
361     536365  12/01/2010 08:26
302     536365  12/01/2010 08:26
31      536366  12/01/2010 08:28
211     53

In [1816]:
# Convertimos la columna InvoiceDate a datetime temporalmente para manejar los valores válidos
retail2_df['InvoiceDate'] = pd.to_datetime(retail2_df['InvoiceDate'], errors='coerce', dayfirst=True)

# Guardamos las fechas originales para las facturas sin número de factura
original_dates = retail2_df[retail2_df['InvoiceNo'] == 0]['InvoiceDate']

# Ordenamos el DataFrame por InvoiceNo para asegurar que las facturas estén en orden
retail2_df = retail2_df.sort_values(by='InvoiceNo')

# Iteramos sobre las filas para corregir los valores inválidos
for i in range(1, len(retail2_df)):
    if (pd.isnull(retail2_df.loc[retail2_df.index[i], 'InvoiceDate']) or 
        retail2_df.loc[retail2_df.index[i], 'InvoiceDate'].year in [1900, 2050]) and \
        retail2_df.loc[retail2_df.index[i], 'InvoiceNo'] != 0:
        
        previous_date = retail2_df.loc[retail2_df.index[i-1], 'InvoiceDate']
        new_date = previous_date + timedelta(minutes=1)
        retail2_df.loc[retail2_df.index[i], 'InvoiceDate'] = new_date

# Aseguramos que las facturas duplicadas tengan la misma hora
retail2_df['InvoiceDate'] = retail2_df.groupby('InvoiceNo')['InvoiceDate'].transform('first')

# Restauramos las fechas originales para las facturas sin número de factura
retail2_df.loc[retail2_df['InvoiceNo'] == 0, 'InvoiceDate'] = original_dates

# Convertimos de nuevo a string para mantener el formato original sin segundos
retail2_df['InvoiceDate'] = retail2_df['InvoiceDate'].dt.strftime('%d/%m/%Y %H:%M')

# Ordenamos el DataFrame por fecha y hora
retail2_df = retail2_df.sort_values(by='InvoiceDate')

# Imprime el resultado de las dos columnas para verificar
print(retail2_df[['InvoiceNo', 'InvoiceDate']])

     InvoiceNo       InvoiceDate
78           0  01/01/2050 00:00
307     536365  12/01/2010 08:26
302     536365  12/01/2010 08:26
47      536365  12/01/2010 08:26
175     536365  12/01/2010 08:26
313     536365  12/01/2010 08:26
38      536365  12/01/2010 08:26
361     536365  12/01/2010 08:26
211     536366  12/01/2010 08:28
31      536366  12/01/2010 08:28
273     536367  12/01/2010 08:34
222     536367  12/01/2010 08:34
282     536367  12/01/2010 08:34
86      536367  12/01/2010 08:34
247     536367  12/01/2010 08:34
388     536367  12/01/2010 08:34
230     536367  12/01/2010 08:34
206     536367  12/01/2010 08:34
360     536367  12/01/2010 08:34
117     536367  12/01/2010 08:34
438     536368  12/01/2010 08:35
72      536368  12/01/2010 08:35
421     536369  12/01/2010 08:36
374     536370  12/01/2010 09:00
267     536370  12/01/2010 09:00
430     536370  12/01/2010 09:00
341     536371  12/01/2010 09:01
383     536371  12/01/2010 09:01
10      536372  12/01/2010 09:02
30      53

In [1817]:
# Imprimir los datos de las columnas especificadas en el índice 312
print(retail2_df.loc[251, ['InvoiceNo', 'Description', 'InvoiceDate']])

InvoiceNo                              0
Description    HAND WARMER RED POLKA DOT
InvoiceDate             12/01/2010 12:53
Name: 251, dtype: object


In [1818]:
# Imprimir los datos de las columnas especificadas en el índice 312
print(retail2_df.loc[208, ['InvoiceNo', 'Description', 'InvoiceDate']])

InvoiceNo                                       0
Description    WHITE HANGING HEART T-LIGHT HOLDER
InvoiceDate                      12/01/2010 13:15
Name: 208, dtype: object


In [1819]:
# Convertimos la columna InvoiceDate a datetime
retail2_df['InvoiceDate'] = pd.to_datetime(retail2_df['InvoiceDate'], format='%d/%m/%Y %H:%M')

# Creamos un diccionario para asignar los valores correctos basados en el índice
index_mapping = {
    
    251: {'InvoiceNo1': 536603, 'InvoiceDate': '12/01/2010 12:53'},
    104: {'InvoiceNo1': 536602, 'InvoiceDate': '12/01/2010 12:52'}
    
}

# Asignamos los valores correctos basados en el índice
for idx, values in index_mapping.items():
    retail2_df.loc[retail2_df.index == idx, 'InvoiceNo'] = values['InvoiceNo1']
    retail2_df.loc[retail2_df.index == idx, 'InvoiceDate'] = pd.to_datetime(values['InvoiceDate'], format='%d/%m/%Y %H:%M')

# Convertimos de nuevo a string para mantener el formato original sin segundos
retail2_df['InvoiceDate'] = retail2_df['InvoiceDate'].dt.strftime('%d/%m/%Y %H:%M')

# Ordenamos el DataFrame por número de factura
retail2_df = retail2_df.sort_values(by='InvoiceNo')

In [1820]:
# Convertimos la columna InvoiceDate a datetime temporalmente para manejar los valores válidos
retail2_df['InvoiceDate'] = pd.to_datetime(retail2_df['InvoiceDate'], errors='coerce', dayfirst=True)

# Ordenamos el DataFrame por fecha y hora para asegurar que las fechas estén en orden
retail2_df = retail2_df.sort_values(by='InvoiceDate')

# Inicializamos el número de factura
current_invoice_no = int(retail2_df['InvoiceNo'].max()) + 1

# Iteramos sobre las filas para asignar números de factura a las filas sin número de factura
for i in range(len(retail2_df)):
    if retail2_df.loc[retail2_df.index[i], 'InvoiceNo'] == 0:
        previous_invoice_no = retail2_df.loc[retail2_df.index[i-1], 'InvoiceNo']
        retail2_df.loc[retail2_df.index[i], 'InvoiceNo'] = previous_invoice_no + 1

# Convertimos de nuevo a string para mantener el formato original sin segundos
retail2_df['InvoiceDate'] = retail2_df['InvoiceDate'].dt.strftime('%d/%m/%Y %H:%M')

# Ordenamos el DataFrame por fecha y hora
retail2_df = retail2_df.sort_values(by='InvoiceDate')

# Imprime el resultado de las dos columnas para verificar
print(retail2_df[['InvoiceNo', 'InvoiceDate']])

     InvoiceNo       InvoiceDate
78      536745  01/01/2050 00:00
302     536365  12/01/2010 08:26
175     536365  12/01/2010 08:26
313     536365  12/01/2010 08:26
47      536365  12/01/2010 08:26
307     536365  12/01/2010 08:26
38      536365  12/01/2010 08:26
361     536365  12/01/2010 08:26
211     536366  12/01/2010 08:28
31      536366  12/01/2010 08:28
117     536367  12/01/2010 08:34
230     536367  12/01/2010 08:34
388     536367  12/01/2010 08:34
222     536367  12/01/2010 08:34
206     536367  12/01/2010 08:34
360     536367  12/01/2010 08:34
273     536367  12/01/2010 08:34
282     536367  12/01/2010 08:34
247     536367  12/01/2010 08:34
86      536367  12/01/2010 08:34
72      536368  12/01/2010 08:35
438     536368  12/01/2010 08:35
421     536369  12/01/2010 08:36
267     536370  12/01/2010 09:00
374     536370  12/01/2010 09:00
430     536370  12/01/2010 09:00
383     536371  12/01/2010 09:01
341     536371  12/01/2010 09:01
10      536372  12/01/2010 09:02
30      53

In [1821]:
# Imprimir los datos de las columnas especificadas en el índice 312
print(retail2_df.loc[104, ['InvoiceNo', 'Description', 'InvoiceDate']])

InvoiceNo                                  536602
Description    WHITE HANGING HEART T-LIGHT HOLDER
InvoiceDate                      12/01/2010 12:52
Name: 104, dtype: object


- Cambio de formatos de fechas

In [1822]:
# Convertimos la columna InvoiceDate a datetime
retail2_df['InvoiceDate'] = pd.to_datetime(retail2_df['InvoiceDate'], format='%d/%m/%Y %H:%M')

# Creamos un diccionario para asignar los valores correctos basados en el índice
index_mapping = {
    78: {'InvoiceNo1': 536653, 'InvoiceDate': '12/01/2010 13:43'},
    52: {'InvoiceNo1': 536730, 'InvoiceDate': '12/01/2010 15:00'},
    412: {'InvoiceNo1': 536384, 'InvoiceDate': '12/01/2010 9:14'}
}

# Asignamos los valores correctos basados en el índice
for idx, values in index_mapping.items():
    retail2_df.loc[retail2_df.index == idx, 'InvoiceNo'] = values['InvoiceNo1']
    retail2_df.loc[retail2_df.index == idx, 'InvoiceDate'] = pd.to_datetime(values['InvoiceDate'], format='%d/%m/%Y %H:%M')

# Convertimos de nuevo a string para mantener el formato original sin segundos
retail2_df['InvoiceDate'] = retail2_df['InvoiceDate'].dt.strftime('%d/%m/%Y %H:%M')

# Ordenamos el DataFrame por número de factura
retail2_df = retail2_df.sort_values(by='InvoiceNo')


In [1823]:
# Identificar los datos inválidos en la columna Description, incluyendo NaN y vacíos
invalid_data = retail2_df[retail2_df['InvoiceDate'].isnull() | (retail2_df['InvoiceDate'] == '') | (retail2_df['InvoiceDate'].str.lower() == 'nan')]

# Mostrar los datos inválidos y la cantidad
print(invalid_data[['InvoiceNo','InvoiceDate']])
print(f"Cantidad de datos inválidos: {invalid_data.shape[0]}")

Empty DataFrame
Columns: [InvoiceNo, InvoiceDate]
Index: []
Cantidad de datos inválidos: 0


In [1824]:
# Imprimir los datos de las columnas especificadas en el índice 251
print(retail2_df.loc[251, ['InvoiceNo', 'Description', 'InvoiceDate']])

InvoiceNo                         536603
Description    HAND WARMER RED POLKA DOT
InvoiceDate             12/01/2010 12:53
Name: 251, dtype: object


## Continuaciónn de la columna InvoiceCode
- Verificación del rango del número de los datos de la columna : Número de factura que empieza hasta que se termina.


- verificacion del rango del número de los datos de la columna 

In [1825]:
# Suponiendo que retail2_df ya está definido
min_invoice = retail2_df['InvoiceNo'].min()
max_invoice = retail2_df['InvoiceNo'].max()

print(f"El número de factura empieza en: {min_invoice}")
print(f"El número de factura finaliza en: {max_invoice}")

El número de factura empieza en: 536365
El número de factura finaliza en: 536744


- Verificación de facturas faltantes entre los rangos  

In [1826]:
expected_range = set(range(536365, 536744))
actual_values = set(retail2_df['InvoiceNo'].dropna().astype(int))
missing_values = expected_range - actual_values

print(f"Valores faltantes en la columna InvoiceNo: {sorted(missing_values)}")

Valores faltantes en la columna InvoiceNo: []


In [1827]:
# Ordenamos el DataFrame por fecha y hora
retail2_df = retail2_df.sort_values(by='InvoiceDate')

# Imprime el resultado de las dos columnas para verificar
print(retail2_df[['InvoiceNo', 'InvoiceDate']])

     InvoiceNo       InvoiceDate
302     536365  12/01/2010 08:26
175     536365  12/01/2010 08:26
313     536365  12/01/2010 08:26
47      536365  12/01/2010 08:26
307     536365  12/01/2010 08:26
38      536365  12/01/2010 08:26
361     536365  12/01/2010 08:26
211     536366  12/01/2010 08:28
31      536366  12/01/2010 08:28
273     536367  12/01/2010 08:34
282     536367  12/01/2010 08:34
117     536367  12/01/2010 08:34
360     536367  12/01/2010 08:34
206     536367  12/01/2010 08:34
247     536367  12/01/2010 08:34
388     536367  12/01/2010 08:34
222     536367  12/01/2010 08:34
86      536367  12/01/2010 08:34
230     536367  12/01/2010 08:34
72      536368  12/01/2010 08:35
438     536368  12/01/2010 08:35
421     536369  12/01/2010 08:36
267     536370  12/01/2010 09:00
430     536370  12/01/2010 09:00
374     536370  12/01/2010 09:00
341     536371  12/01/2010 09:01
383     536371  12/01/2010 09:01
10      536372  12/01/2010 09:02
30      536373  12/01/2010 09:03
142     53

## Columna Quantity

- Imprimir valores únicos.

In [1828]:
retail2_df = retail2_df.sort_values(by='Quantity')
print(retail2_df['Quantity'].unique() )

['-20' '-3' '-6' '-60' '10' '100' '12' '15' '2' '20' '200' '3' '32' '4'
 '5' '6' '60' '8' 'unknown']


In [1829]:
retail2_df = retail2_df.sort_values(by='Description')
print(retail2_df[['Description','Quantity']])

                             Description Quantity
318        ASSORTED COLOUR BIRD ORNAMENT       32
257        ASSORTED COLOUR BIRD ORNAMENT       32
28         ASSORTED COLOUR BIRD ORNAMENT       32
80         ASSORTED COLOUR BIRD ORNAMENT  unknown
360        ASSORTED COLOUR BIRD ORNAMENT       32
352        ASSORTED COLOUR BIRD ORNAMENT       32
96         ASSORTED COLOUR BIRD ORNAMENT       32
189        ASSORTED COLOUR BIRD ORNAMENT       32
389         BAKING SET 9 PIECE RETROSPOT        6
296         BAKING SET 9 PIECE RETROSPOT        6
39          BAKING SET 9 PIECE RETROSPOT        6
8           BAKING SET 9 PIECE RETROSPOT        6
61          BAKING SET 9 PIECE RETROSPOT        6
380         BAKING SET 9 PIECE RETROSPOT        6
228         BAKING SET 9 PIECE RETROSPOT        6
221         BAKING SET 9 PIECE RETROSPOT        6
4           BAKING SET 9 PIECE RETROSPOT        6
64          BAKING SET 9 PIECE RETROSPOT        6
70          BAKING SET 9 PIECE RETROSPOT        6


# Reemplazar valores "unknown" por la cantidad más común para cada descripción

In [1830]:
# Reemplazar valores "unknown" por la cantidad más común para cada descripción
def replace_unknown(row):
    if row['Quantity'] == 'unknown':
        most_common_quantity = retail2_df[retail2_df['Description'] == row['Description']]['Quantity'].mode()[0]
        return most_common_quantity
    return row['Quantity']

retail2_df['Quantity'] = retail2_df.apply(replace_unknown, axis=1)

# Convertir la columna Quantity a tipo numérico
retail2_df['Quantity'] = pd.to_numeric(retail2_df['Quantity'])

# Verificar y corregir otras cantidades negativas (si es necesario)
# Aquí puedes decidir cómo manejar otras cantidades negativas
# Por ejemplo, si todas las cantidades negativas deben ser corregidas a positivas:
# retail2_df['Quantity'] = retail2_df['Quantity'].abs()

print(retail2_df[['Description','Quantity']].head())
 

         

                       Description  Quantity
318  ASSORTED COLOUR BIRD ORNAMENT        32
257  ASSORTED COLOUR BIRD ORNAMENT        32
28   ASSORTED COLOUR BIRD ORNAMENT        32
80   ASSORTED COLOUR BIRD ORNAMENT        32
360  ASSORTED COLOUR BIRD ORNAMENT        32


In [1831]:
print(retail2_df['Quantity'].dtype)

int64


- Vamos a  encontrar la cantidad más frecuente para cada descripción en un DataFrame, manteniendo las cantidades negativas tal cual.

In [1832]:
# Función para encontrar la cantidad que más veces se repite por descripción
def most_frequent_quantity(group):
    positive_quantities = group[group >= 0]
    if not positive_quantities.empty:
        return positive_quantities.mode()[0]
    return group.mode()[0]

# Aplicar la función a cada grupo de descripción
most_frequent_quantities = retail2_df[retail2_df['Quantity'] >= 0].groupby('Description')['Quantity'].transform(lambda x: x.mode()[0])

# Mantener las devoluciones (cantidades negativas) tal cual
retail2_df['Quantity'] = retail2_df.apply(lambda row: row['Quantity'] if row['Quantity'] < 0 else most_frequent_quantities[row.name], axis=1)

print(retail2_df[['Description', 'Quantity']])

                             Description  Quantity
318        ASSORTED COLOUR BIRD ORNAMENT        32
257        ASSORTED COLOUR BIRD ORNAMENT        32
28         ASSORTED COLOUR BIRD ORNAMENT        32
80         ASSORTED COLOUR BIRD ORNAMENT        32
360        ASSORTED COLOUR BIRD ORNAMENT        32
352        ASSORTED COLOUR BIRD ORNAMENT        32
96         ASSORTED COLOUR BIRD ORNAMENT        32
189        ASSORTED COLOUR BIRD ORNAMENT        32
389         BAKING SET 9 PIECE RETROSPOT         6
296         BAKING SET 9 PIECE RETROSPOT         6
39          BAKING SET 9 PIECE RETROSPOT         6
8           BAKING SET 9 PIECE RETROSPOT         6
61          BAKING SET 9 PIECE RETROSPOT         6
380         BAKING SET 9 PIECE RETROSPOT         6
228         BAKING SET 9 PIECE RETROSPOT         6
221         BAKING SET 9 PIECE RETROSPOT         6
4           BAKING SET 9 PIECE RETROSPOT         6
64          BAKING SET 9 PIECE RETROSPOT         6
70          BAKING SET 9 PIECE 

In [1833]:
# Corregir el valor específico en el índice 233
retail2_df.loc[233, 'Quantity'] = -6


En la data el indice 233 no hizo el cambio correcto en el valor de la cantidad en la columna Quantity 

In [1834]:
# Imprimir los datos de las columnas especificadas en el índice 233
print(retail2_df.loc[233, ['Description', 'Quantity']])

Description    WHITE HANGING HEART T-LIGHT HOLDER
Quantity                                       -6
Name: 233, dtype: object


In [1835]:
print(retail2_df['Quantity'].unique())

[ 32   6  12   2   3  -3  10   8  -6  20 200  15   4 -20]


# Columna UnitPrice

Vamos a convertir los valores de UnitPrice a float, manejar los valores ‘unknown’ como NaN, encontrar el precio más frecuente por descripción, y convertir los valores enteros de tres cifras a decimales. Aquí tienes un ejemplo de cómo se vería la salida después de aplicar todas estas transformaciones:

In [1836]:
# Convertir los valores de 'UnitPrice' a float, manejando 'unknown' como NaN
retail2_df['UnitPrice'] = pd.to_numeric(retail2_df['UnitPrice'], errors='coerce')

# Función para encontrar el precio que más veces se repite por descripción
def most_frequent_price(group):
    known_prices = group.dropna()
    if not known_prices.empty:
        return known_prices.mode()[0]
    return np.nan

# Aplicar la función a cada grupo de descripción
most_frequent_prices = retail2_df.groupby('Description')['UnitPrice'].transform(most_frequent_price)

# Reemplazar los valores NaN con el precio más frecuente
retail2_df['UnitPrice'] = retail2_df['UnitPrice'].fillna(most_frequent_prices)

# Función para convertir los valores enteros de tres cifras a decimales
def convert_to_decimal(price):
    if price >= 100:
        return price / 100
    return price

# Aplicar la función a la columna 'UnitPrice'
retail2_df['UnitPrice'] = retail2_df['UnitPrice'].apply(convert_to_decimal)

# Verificar los resultados
print(retail2_df[['Description', 'UnitPrice']])

                             Description  UnitPrice
318        ASSORTED COLOUR BIRD ORNAMENT       1.69
257        ASSORTED COLOUR BIRD ORNAMENT       1.69
28         ASSORTED COLOUR BIRD ORNAMENT       1.69
80         ASSORTED COLOUR BIRD ORNAMENT       1.69
360        ASSORTED COLOUR BIRD ORNAMENT       1.69
352        ASSORTED COLOUR BIRD ORNAMENT       1.69
96         ASSORTED COLOUR BIRD ORNAMENT       1.69
189        ASSORTED COLOUR BIRD ORNAMENT       1.69
389         BAKING SET 9 PIECE RETROSPOT       4.95
296         BAKING SET 9 PIECE RETROSPOT       4.95
39          BAKING SET 9 PIECE RETROSPOT       4.95
8           BAKING SET 9 PIECE RETROSPOT       4.95
61          BAKING SET 9 PIECE RETROSPOT       4.95
380         BAKING SET 9 PIECE RETROSPOT       4.95
228         BAKING SET 9 PIECE RETROSPOT       4.95
221         BAKING SET 9 PIECE RETROSPOT       4.95
4           BAKING SET 9 PIECE RETROSPOT       4.95
64          BAKING SET 9 PIECE RETROSPOT       4.95
70          

In [1837]:
retail2_df['UnitPrice'] = retail2_df['UnitPrice'].astype(float)
print(retail2_df['UnitPrice'].dtype)

float64


In [1838]:
# Identificar los datos inválidos en la columna 'UnitPrice', incluyendo NaN
invalid_data = retail2_df[retail2_df['UnitPrice'].isnull() | (retail2_df['UnitPrice'] == 0)]

# Mostrar los datos inválidos y la cantidad
print(invalid_data[['UnitPrice']])
print(f"Cantidad de datos inválidos: {invalid_data.shape[0]}")

Empty DataFrame
Columns: [UnitPrice]
Index: []
Cantidad de datos inválidos: 0


## Columna Country

In [1839]:
print(retail2_df['Country'].dtype)

object


Vamos a unificar criterios deacuerdo la ubicacion o relacion del nombre con cada país

In [1840]:
# Función para normalizar los valores de la columna 'Country'
def normalize_country(country):
    country = str(country).strip().lower()
    if re.search(r'denmark', country):
        return 'Denmark'
    elif re.search(r'united\s?kingdom|u\.k\.|england', country):
        return 'United Kingdom'
    elif re.search(r'germany', country):
        return 'Germany'
    else:
        return ''

# Convertir todos los valores de la columna 'Country' a cadenas
retail2_df['Country'] = retail2_df['Country'].astype(str)

# Aplicar la función de normalización
retail2_df['Country'] = retail2_df['Country'].apply(normalize_country)

# Verificar los resultados
print(retail2_df['Country'].unique())

['Denmark' 'Germany' 'United Kingdom' '']


In [1841]:
# Identificar los datos inválidos en la columna Country, incluyendo NaN y vacíos
invalid_data = retail2_df[retail2_df['Country'].isnull() | (retail2_df['Country'] == '') | (retail2_df['Country'].str.lower() == 'nan')]

# Mostrar los datos inválidos y la cantidad
print(invalid_data[['Country']])
print(f"Cantidad de datos inválidos: {invalid_data.shape[0]}")

    Country
382        
179        
269        
16         
66         
392        
264        
322        
404        
98         
Cantidad de datos inválidos: 10


In [1842]:
# Lista de índices inválidos
invalid_indices = [382, 179, 269, 16, 66, 392, 264, 322, 404, 98]

# Filtrar los datos en los índices inválidos
invalid_data = retail2_df.loc[invalid_indices]

print(invalid_data)

     InvoiceNo StockCode                          Description  Quantity  \
382     536680     22622       BOX OF VINTAGE ALPHABET BLOCKS         2   
179     536396     22623         BOX OF VINTAGE JIGSAW BLOCKS         3   
269     536547     22112           CHOCOLATE HOT WATER BOTTLE        10   
16      536676     22749    FELTCRAFT PRINCESS CHARLOTTE DOLL         8   
66      536567    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
392     536523     21724                 RED HARMONICA IN BOX        10   
264     536461     22111            SCANDINAVIAN REDS RIBBONS        10   
322     536379     22111            SCANDINAVIAN REDS RIBBONS        10   
404     536622     22111            SCANDINAVIAN REDS RIBBONS        10   
98      536500     21723   STRIPED CHARLIE+LOLA CHARLOTTE BAG        20   

          InvoiceDate  UnitPrice  CustomerID Country    CustomerName  \
382  12/01/2010 14:10       9.95       14396            Eva Williams   
179  12/01/2010 09:26       4.

In [1843]:
# Información de los índices y países
data_to_update = {
    269: 'Germany',
    179: 'Denmark',
    98: 'United Kingdom',
    392: 'United Kingdom',
    66: 'United Kingdom',
    264: 'Denmark',
    382: 'Denmark',
    322: 'United Kingdom',
    404: 'Denmark',
    16: 'Germany'
}

# Actualizar la columna 'Country' en los índices especificados
for index, country in data_to_update.items():
    retail2_df.at[index, 'Country'] = country

print(retail2_df.loc[list(data_to_update.keys())])

     InvoiceNo StockCode                          Description  Quantity  \
269     536547     22112           CHOCOLATE HOT WATER BOTTLE        10   
179     536396     22623         BOX OF VINTAGE JIGSAW BLOCKS         3   
98      536500     21723   STRIPED CHARLIE+LOLA CHARLOTTE BAG        20   
392     536523     21724                 RED HARMONICA IN BOX        10   
66      536567    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
264     536461     22111            SCANDINAVIAN REDS RIBBONS        10   
382     536680     22622       BOX OF VINTAGE ALPHABET BLOCKS         2   
322     536379     22111            SCANDINAVIAN REDS RIBBONS        10   
404     536622     22111            SCANDINAVIAN REDS RIBBONS        10   
16      536676     22749    FELTCRAFT PRINCESS CHARLOTTE DOLL         8   

          InvoiceDate  UnitPrice  CustomerID         Country    CustomerName  \
269  12/01/2010 11:57       3.39       11855         Germany   David Johnson   
179  12/01/201

## Columna Address

In [1844]:
# Imprimir los datos de las columnas especificadas en el índice 312
print(retail2_df.loc[47, ['InvoiceNo','Description', 'Quantity']])

InvoiceNo                                   536365
Description    KNITTED UNION FLAG HOT WATER BOTTLE
Quantity                                         6
Name: 47, dtype: object


In [1845]:
# Agrupar por 'InvoiceNo' y encontrar el 'CustomerName' más frecuente
retail2_df['FrequentCustomer'] = retail2_df.groupby('InvoiceNo')['CustomerName'].transform(lambda x: x.mode()[0])

In [1846]:
# Ordenar el DataFrame por 'InvoiceNo'
retail2_df = retail2_df.sort_values(by='InvoiceNo')

# Imprimir las columnas relacionadas
print(retail2_df[['InvoiceNo', 'CustomerName', 'FrequentCustomer']])

     InvoiceNo      CustomerName  FrequentCustomer
307     536365       Frank Brown       Alice Smith
302     536365       Jane Taylor       Alice Smith
38      536365       Eva Johnson       Alice Smith
313     536365       Grace Moore       Alice Smith
175     536365         Eva Davis       Alice Smith
361     536365        John Jones       Alice Smith
47      536365       Alice Smith       Alice Smith
31      536366       David Moore       David Moore
211     536366      Grace Wilson       David Moore
117     536367       Grace Smith       Alice Jones
86      536367        Jane Jones       Alice Jones
222     536367       David Moore       Alice Jones
273     536367     Grace Johnson       Alice Jones
388     536367       Jane Wilson       Alice Jones
247     536367      Henry Wilson       Alice Jones
282     536367       Henry Jones       Alice Jones
230     536367        John Smith       Alice Jones
206     536367       Alice Jones       Alice Jones
360     536367        Jane Moor

In [1847]:
# Agrupar por 'FrequentCustomer' y encontrar el 'Country' más frecuente
retail2_df['FrequentCountry'] = retail2_df.groupby('FrequentCustomer')['Country'].transform(lambda x: x.mode()[0])

# Ordenar el DataFrame por 'InvoiceNo'
retail2_df = retail2_df.sort_values(by='InvoiceNo')

# Imprimir las columnas relacionadas
print(retail2_df[['InvoiceNo', 'CustomerName', 'FrequentCustomer', 'FrequentCountry']])

     InvoiceNo      CustomerName  FrequentCustomer FrequentCountry
307     536365       Frank Brown       Alice Smith         Denmark
302     536365       Jane Taylor       Alice Smith         Denmark
38      536365       Eva Johnson       Alice Smith         Denmark
313     536365       Grace Moore       Alice Smith         Denmark
175     536365         Eva Davis       Alice Smith         Denmark
361     536365        John Jones       Alice Smith         Denmark
47      536365       Alice Smith       Alice Smith         Denmark
31      536366       David Moore       David Moore         Germany
211     536366      Grace Wilson       David Moore         Germany
206     536367       Alice Jones       Alice Jones         Denmark
273     536367     Grace Johnson       Alice Jones         Denmark
86      536367        Jane Jones       Alice Jones         Denmark
282     536367       Henry Jones       Alice Jones         Denmark
388     536367       Jane Wilson       Alice Jones         Den

In [1848]:
# Agrupar por 'FrequentCustomer' y encontrar el 'Email' más frecuente
retail2_df['FrequentEmail'] = retail2_df.groupby('FrequentCustomer')['Email'].transform(lambda x: x.mode()[0])

# Ordenar el DataFrame por 'InvoiceNo'
retail2_df = retail2_df.sort_values(by='InvoiceNo')

# Imprimir las columnas relacionadas
print(retail2_df[['InvoiceNo', 'CustomerName', 'FrequentCustomer', 'FrequentEmail']])

     InvoiceNo      CustomerName  FrequentCustomer               FrequentEmail
307     536365       Frank Brown       Alice Smith     alice.smith@example.com
302     536365       Jane Taylor       Alice Smith     alice.smith@example.com
38      536365       Eva Johnson       Alice Smith     alice.smith@example.com
313     536365       Grace Moore       Alice Smith     alice.smith@example.com
175     536365         Eva Davis       Alice Smith     alice.smith@example.com
361     536365        John Jones       Alice Smith     alice.smith@example.com
47      536365       Alice Smith       Alice Smith     alice.smith@example.com
31      536366       David Moore       David Moore        david.moore@mail.com
211     536366      Grace Wilson       David Moore        david.moore@mail.com
117     536367       Grace Smith       Alice Jones        alice.jones@demo.net
282     536367       Henry Jones       Alice Jones        alice.jones@demo.net
273     536367     Grace Johnson       Alice Jones  

In [1849]:
# Agrupar por 'FrequentCustomer' y encontrar el 'PhoneNumber' más frecuente
retail2_df['FrequentPhone'] = retail2_df.groupby('FrequentCustomer')['PhoneNumber'].transform(lambda x: x.mode()[0])

# Ordenar el DataFrame por 'InvoiceNo'
retail2_df = retail2_df.sort_values(by='InvoiceNo')

# Imprimir las columnas relacionadas
print(retail2_df[['InvoiceNo', 'CustomerName', 'FrequentCustomer', 'FrequentEmail','FrequentPhone']])

     InvoiceNo      CustomerName  FrequentCustomer  \
307     536365       Frank Brown       Alice Smith   
302     536365       Jane Taylor       Alice Smith   
38      536365       Eva Johnson       Alice Smith   
313     536365       Grace Moore       Alice Smith   
175     536365         Eva Davis       Alice Smith   
361     536365        John Jones       Alice Smith   
47      536365       Alice Smith       Alice Smith   
31      536366       David Moore       David Moore   
211     536366      Grace Wilson       David Moore   
206     536367       Alice Jones       Alice Jones   
247     536367      Henry Wilson       Alice Jones   
282     536367       Henry Jones       Alice Jones   
230     536367        John Smith       Alice Jones   
388     536367       Jane Wilson       Alice Jones   
222     536367       David Moore       Alice Jones   
360     536367        Jane Moore       Alice Jones   
86      536367        Jane Jones       Alice Jones   
117     536367       Grace S

In [1850]:
# Diccionario de categorías basadas en las descripciones
category_mapping = {
    'ASSORTED COLOUR BIRD ORNAMENT': 'Home & Garden',
    'BAKING SET 9 PIECE RETROSPOT': 'Home & Garden',
    'BLUE HARMONICA IN BOX': 'Toys',
    'BOX OF 6 ASSORTED COLOUR TEASPOONS': 'Home & Garden',
    'BOX OF VINTAGE ALPHABET BLOCKS': 'Toys',
    'BOX OF VINTAGE JIGSAW BLOCKS': 'Toys',
    'CHOCOLATE HOT WATER BOTTLE': 'Beauty',
    'COFFEE MUG APPLES DESIGN': 'Home & Garden',
    'CREAM CUPID HEARTS COAT HANGER': 'Home & Garden',
    'DOORMAT NEW ENGLAND': 'Home & Garden',
    'FELTCRAFT PRINCESS CHARLOTTE DOLL': 'Toys',
    'GLASS JAR DAISY FRESH COTTON WOOL': 'Home & Garden',
    'GLASS STAR FROSTED TLIGHT HOLDER': 'Home & Garden',
    'HAND WARMER RED POLKA DOT': 'Clothing',
    'HAND WARMER UNION JACK': 'Clothing',
    'HANGING STAR WITH BELLS': 'Home & Garden',
    'HEART OF WICKER LARGE': 'Home & Garden',
    'HOME BUILDING BLOCK WORD': 'Home & Garden',
    'IVORY KNITTED MUG COSY': 'Home & Garden',
    'JAM MAKING SET WITH JARS': 'Home & Garden',
    'JUMBO BAG PINK POLKADOT': 'Clothing',
    'JUMBO BAG RED RETROSPOT': 'Clothing',
    'KNITTED UNION FLAG HOT WATER BOTTLE': 'Beauty',
    'LOVE BUILDING BLOCK WORD': 'Home & Garden',
    'PACK OF 12 BLUE RETROSPOT TISSUES': 'Home & Garden',
    'PACK OF 12 PINK POLKADOT TISSUES': 'Home & Garden',
    'PACK OF 12 RED RETROSPOT TISSUES': 'Home & Garden',
    'PACK OF 6 SMALL FRUIT STRAWS': 'Home & Garden',
    'POPPYS PLAYHOUSE BEDROOM': 'Toys',
    'POPPYS PLAYHOUSE KITCHEN': 'Toys',
    'RECIPE BOX WITH METAL HEART': 'Home & Garden',
    'RED GLASS CANDLE HOLDER STAR': 'Home & Garden',
    'RED HARMONICA IN BOX': 'Toys',
    'RED TOADSTOOL LED NIGHT LIGHT': 'Home & Garden',
    'RED WOOLLY HOTTIE WHITE HEART': 'Beauty',
    'RETROSPOT TEA SET CERAMIC 11 PC': 'Home & Garden',
    'SCANDINAVIAN REDS RIBBONS': 'Home & Garden',
    'SET 2 TEA TOWELS I LOVE LONDON': 'Home & Garden',
    'SET 7 BABUSHKA NESTING BOXES': 'Toys',
    'SET OF 3 CAKE TINS PANTRY DESIGN': 'Home & Garden',
    'SET OF 7 BABUSHKA NESTING BOXES': 'Toys',
    'STRIPED CHARLIE+LOLA CHARLOTTE BAG': 'Clothing',
    'WHITE HANGING HEART T-LIGHT HOLDER': 'Home & Garden',
    'WHITE METAL LANTERN': 'Home & Garden'
}

# Actualizar la columna 'Category' basada en las descripciones
retail2_df['Category'] = retail2_df['Description'].map(category_mapping)

# Ordenar el DataFrame por 'InvoiceNo'
retail2_df = retail2_df.sort_values(by='Description')

# Imprimir las columnas relacionadas
print(retail2_df[['Description', 'Category']].head(20))

                       Description       Category
352  ASSORTED COLOUR BIRD ORNAMENT  Home & Garden
257  ASSORTED COLOUR BIRD ORNAMENT  Home & Garden
28   ASSORTED COLOUR BIRD ORNAMENT  Home & Garden
318  ASSORTED COLOUR BIRD ORNAMENT  Home & Garden
360  ASSORTED COLOUR BIRD ORNAMENT  Home & Garden
96   ASSORTED COLOUR BIRD ORNAMENT  Home & Garden
189  ASSORTED COLOUR BIRD ORNAMENT  Home & Garden
80   ASSORTED COLOUR BIRD ORNAMENT  Home & Garden
221   BAKING SET 9 PIECE RETROSPOT  Home & Garden
61    BAKING SET 9 PIECE RETROSPOT  Home & Garden
246   BAKING SET 9 PIECE RETROSPOT  Home & Garden
64    BAKING SET 9 PIECE RETROSPOT  Home & Garden
228   BAKING SET 9 PIECE RETROSPOT  Home & Garden
296   BAKING SET 9 PIECE RETROSPOT  Home & Garden
279   BAKING SET 9 PIECE RETROSPOT  Home & Garden
168   BAKING SET 9 PIECE RETROSPOT  Home & Garden
43    BAKING SET 9 PIECE RETROSPOT  Home & Garden
389   BAKING SET 9 PIECE RETROSPOT  Home & Garden
287   BAKING SET 9 PIECE RETROSPOT  Home & Garden


In [1851]:
retail2_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,CustomerName,Email,Address,PhoneNumber,Category,Supplier,StockLevel,Discount,SaleChannel,ReturnStatus,ProductWeight,ProductDimensions,ShippingCost,SalesRegion,PromotionCode,PaymentMethod,FrequentCustomer,FrequentCountry,FrequentEmail,FrequentPhone
352,536473,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/01/2010 10:43,1.69,18345,Denmark,Alice Taylor,alice.taylor@test.org,450 Ash St,555-440-9320,Home & Garden,Supplier D,840,3.81,Online,Not Returned,6.62,25x98x76 cm,16.34,South America,PROMO20,Bank Transfer,Alice Taylor,Denmark,alice.taylor@example.com,555-113-5599
257,536573,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/01/2010 12:23,1.69,12972,Germany,Henry Williams,henry.williams@example.com,594 Market St,555-403-9125,Home & Garden,Supplier E,645,26.97,Online,Returned,6.99,29x98x87 cm,6.83,North America,PROMO10,Bank Transfer,Henry Williams,Denmark,henry.williams@example.com,555-150-8776
28,536534,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/01/2010 11:44,1.69,17866,United Kingdom,Grace Moore,grace.moore@demo.net,390 Broadway,555-615-8266,Home & Garden,Supplier A,106,41.94,In-Store,Returned,5.96,90x8x58 cm,15.71,Australia,SALE15,Gift Card,Grace Moore,United Kingdom,grace.moore@demo.net,555-363-8970
318,536673,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/01/2010 14:03,1.69,16432,Denmark,Grace Johnson,grace.johnson@test.org,815 Maple St,555-830-5263,Home & Garden,Supplier A,930,48.46,Online,Returned,1.65,32x96x99 cm,9.11,Asia,PROMO20,PayPal,Grace Johnson,Denmark,grace.johnson@example.com,555-174-7981
360,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/01/2010 08:34,1.69,11306,Denmark,Jane Moore,jane.moore@example.com,709 Pine St,555-775-9378,Home & Garden,Supplier A,865,28.29,Online,Not Returned,3.94,77x28x69 cm,19.89,South America,DISCOUNT5,Cash,Alice Jones,Denmark,alice.jones@demo.net,555-123-9595


## Columna PromotionCode

- Se organiza es columna tomando como criterio las promociones más frecuentes para cada descripcion (Description) de producto.
- Completamos valores faltantes.
 

- Se organiza es columna tomando como criterio las promociones más frecuentes para cada descripcion (Description) de producto.

In [1852]:
# Calcular el PromotionCode más frecuente para cada descripción
promotion_code_mapping = retail2_df.groupby('Description')['PromotionCode'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None).to_dict()

# Asignar el PromotionCode más frecuente a la columna 'PromotionCode' para cada descripción
retail2_df['PromotionCode'] = retail2_df['Description'].map(promotion_code_mapping)

# Ordenar el DataFrame por 'Description'
retail2_df = retail2_df.sort_values(by='Description')

# Mostrar las primeras filas de las columnas 'Description' y 'PromotionCode'
print(retail2_df[['Description', 'PromotionCode']].head(20))

                       Description PromotionCode
352  ASSORTED COLOUR BIRD ORNAMENT     DISCOUNT5
257  ASSORTED COLOUR BIRD ORNAMENT     DISCOUNT5
28   ASSORTED COLOUR BIRD ORNAMENT     DISCOUNT5
318  ASSORTED COLOUR BIRD ORNAMENT     DISCOUNT5
360  ASSORTED COLOUR BIRD ORNAMENT     DISCOUNT5
96   ASSORTED COLOUR BIRD ORNAMENT     DISCOUNT5
189  ASSORTED COLOUR BIRD ORNAMENT     DISCOUNT5
80   ASSORTED COLOUR BIRD ORNAMENT     DISCOUNT5
167   BAKING SET 9 PIECE RETROSPOT       PROMO10
70    BAKING SET 9 PIECE RETROSPOT       PROMO10
258   BAKING SET 9 PIECE RETROSPOT       PROMO10
380   BAKING SET 9 PIECE RETROSPOT       PROMO10
39    BAKING SET 9 PIECE RETROSPOT       PROMO10
4     BAKING SET 9 PIECE RETROSPOT       PROMO10
8     BAKING SET 9 PIECE RETROSPOT       PROMO10
391   BAKING SET 9 PIECE RETROSPOT       PROMO10
425   BAKING SET 9 PIECE RETROSPOT       PROMO10
319   BAKING SET 9 PIECE RETROSPOT       PROMO10
85    BAKING SET 9 PIECE RETROSPOT       PROMO10
389   BAKING SET 9 P

- Completamos valores faltantes.

In [1853]:
# Información de los índices y códigos de promoción
promo_data_to_update = {
    385: 'PROMO10',
    106: 'PROMO10'
}

# Actualizar la columna 'PromotionCode' en los índices especificados
for index, promo_code in promo_data_to_update.items():
    retail2_df.at[index, 'PromotionCode'] = promo_code

print(retail2_df['PromotionCode'].head())

352    DISCOUNT5
257    DISCOUNT5
28     DISCOUNT5
318    DISCOUNT5
360    DISCOUNT5
Name: PromotionCode, dtype: object


In [1854]:
# Ordenar el DataFrame por 'InvoiceNo'
retail2_df = retail2_df.sort_values(by='Description')
print(retail2_df[['Description','PromotionCode']].head(20))


                       Description PromotionCode
352  ASSORTED COLOUR BIRD ORNAMENT     DISCOUNT5
257  ASSORTED COLOUR BIRD ORNAMENT     DISCOUNT5
28   ASSORTED COLOUR BIRD ORNAMENT     DISCOUNT5
318  ASSORTED COLOUR BIRD ORNAMENT     DISCOUNT5
360  ASSORTED COLOUR BIRD ORNAMENT     DISCOUNT5
96   ASSORTED COLOUR BIRD ORNAMENT     DISCOUNT5
189  ASSORTED COLOUR BIRD ORNAMENT     DISCOUNT5
80   ASSORTED COLOUR BIRD ORNAMENT     DISCOUNT5
221   BAKING SET 9 PIECE RETROSPOT       PROMO10
61    BAKING SET 9 PIECE RETROSPOT       PROMO10
228   BAKING SET 9 PIECE RETROSPOT       PROMO10
246   BAKING SET 9 PIECE RETROSPOT       PROMO10
296   BAKING SET 9 PIECE RETROSPOT       PROMO10
168   BAKING SET 9 PIECE RETROSPOT       PROMO10
43    BAKING SET 9 PIECE RETROSPOT       PROMO10
287   BAKING SET 9 PIECE RETROSPOT       PROMO10
279   BAKING SET 9 PIECE RETROSPOT       PROMO10
389   BAKING SET 9 PIECE RETROSPOT       PROMO10
64    BAKING SET 9 PIECE RETROSPOT       PROMO10
319   BAKING SET 9 P

 - verificar que sea tipod e dato float   

In [1855]:
print(retail2_df['PromotionCode'].dtype)

object


## PaymentMethod

- Se organiza es columna tomando como criterio el método de pago más frecuentes para cada InvoiceNo de producto.




In [1856]:
# Calcular el PaymentMethod más frecuente para cada InvoiceNo
payment_method_mapping = retail2_df.groupby('InvoiceNo')['PaymentMethod'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None).to_dict()

# Asignar el PaymentMethod más frecuente a la columna 'PaymentMethod' para cada InvoiceNo
retail2_df['PaymentMethod'] = retail2_df['InvoiceNo'].map(payment_method_mapping)

# Ordenar el DataFrame por 'InvoiceNo'
retail2_df = retail2_df.sort_values(by='InvoiceNo')

# Mostrar las primeras filas de las columnas 'InvoiceNo' y 'PaymentMethod'
print(retail2_df[['InvoiceNo', 'PaymentMethod']].head(20))

     InvoiceNo PaymentMethod
38      536365     Gift Card
302     536365     Gift Card
175     536365     Gift Card
313     536365     Gift Card
307     536365     Gift Card
361     536365     Gift Card
47      536365     Gift Card
211     536366          Cash
31      536366          Cash
273     536367   Credit Card
117     536367   Credit Card
282     536367   Credit Card
388     536367   Credit Card
206     536367   Credit Card
86      536367   Credit Card
247     536367   Credit Card
230     536367   Credit Card
222     536367   Credit Card
360     536367   Credit Card
72      536368     Gift Card


## Columna ShippingCost

- Se asigna la región de ventas más frecuente a cada InvoiceNo en la columna SalesRegion

In [1857]:
# Calcular el ShippingCost más frecuente para cada descripción
shipping_cost_mapping = retail2_df.groupby('Description')['ShippingCost'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None).to_dict()

# Asignar el ShippingCost más frecuente a la columna 'ShippingCost' para cada descripción
retail2_df['ShippingCost'] = retail2_df['Description'].map(shipping_cost_mapping)

# Mostrar las primeras filas de las columnas 'Description' y 'ShippingCost'
print(retail2_df[['Description', 'ShippingCost']].head(20))

                             Description  ShippingCost
38         RED WOOLLY HOTTIE WHITE HEART          5.87
302         SET 7 BABUSHKA NESTING BOXES          5.60
175     GLASS STAR FROSTED TLIGHT HOLDER          6.44
313       CREAM CUPID HEARTS COAT HANGER          7.94
307   WHITE HANGING HEART T-LIGHT HOLDER          5.27
361                  WHITE METAL LANTERN          5.11
47   KNITTED UNION FLAG HOT WATER BOTTLE          5.72
211            HAND WARMER RED POLKA DOT          5.50
31                HAND WARMER UNION JACK          5.07
273    FELTCRAFT PRINCESS CHARLOTTE DOLL          6.02
117             POPPYS PLAYHOUSE KITCHEN          5.90
282   BOX OF 6 ASSORTED COLOUR TEASPOONS          7.14
388         BOX OF VINTAGE JIGSAW BLOCKS          7.71
206             LOVE BUILDING BLOCK WORD          5.09
86              POPPYS PLAYHOUSE BEDROOM          6.43
247       BOX OF VINTAGE ALPHABET BLOCKS          8.56
230             HOME BUILDING BLOCK WORD          7.51
222       

## Columna SalesRegion

- 


In [1858]:
# Primero, calculamos la región de ventas más frecuente para cada InvoiceNo
frequent_sales_region = retail2_df.groupby('InvoiceNo')['SalesRegion'].agg(lambda x: x.value_counts().idxmax())

# Luego, asignamos esta región de ventas más frecuente a cada InvoiceNo en el dataframe original
retail2_df['SalesRegion'] = retail2_df['InvoiceNo'].map(frequent_sales_region)

# Ordenar el DataFrame por 'InvoiceNo'
retail2_df = retail2_df.sort_values(by='InvoiceNo')

# Mostrar las primeras filas de las columnas 'Description' y 'ShippingCost'
print(retail2_df[['InvoiceNo', 'SalesRegion']].head(20))

     InvoiceNo    SalesRegion
38      536365      Australia
302     536365      Australia
175     536365      Australia
313     536365      Australia
307     536365      Australia
361     536365      Australia
47      536365      Australia
211     536366           Asia
31      536366           Asia
222     536367      Australia
388     536367      Australia
117     536367      Australia
247     536367      Australia
206     536367      Australia
230     536367      Australia
86      536367      Australia
360     536367      Australia
273     536367      Australia
282     536367      Australia
72      536368  North America


## Columna ProductDimensions

- Calculamos las dimensiones más frecuentes para cada Description

In [1859]:
# Primero, calculamos las dimensiones más frecuentes para cada Description
frequent_dimensions = retail2_df.groupby('Description')['ProductDimensions'].agg(lambda x: x.value_counts().idxmax())

# Luego, asignamos estas dimensiones más frecuentes a cada Description en el dataframe original
retail2_df['ProductDimensions'] = retail2_df['Description'].map(frequent_dimensions)

# Ordenar el DataFrame por 'InvoiceNo'
retail2_df = retail2_df.sort_values(by='Description')

# Verificamos el resultado
# Mostrar las primeras filas de las columnas 'Description' y 'ProductDimensions'
print(retail2_df[['Description', 'ProductDimensions']].head(10))

                       Description ProductDimensions
352  ASSORTED COLOUR BIRD ORNAMENT       77x28x69 cm
28   ASSORTED COLOUR BIRD ORNAMENT       77x28x69 cm
257  ASSORTED COLOUR BIRD ORNAMENT       77x28x69 cm
318  ASSORTED COLOUR BIRD ORNAMENT       77x28x69 cm
360  ASSORTED COLOUR BIRD ORNAMENT       77x28x69 cm
96   ASSORTED COLOUR BIRD ORNAMENT       77x28x69 cm
189  ASSORTED COLOUR BIRD ORNAMENT       77x28x69 cm
80   ASSORTED COLOUR BIRD ORNAMENT       77x28x69 cm
221   BAKING SET 9 PIECE RETROSPOT       39x46x29 cm
258   BAKING SET 9 PIECE RETROSPOT       39x46x29 cm


## Columna ProductWeight

- Calculamos el peso más frecuente para cada Description



In [1860]:

# Primero, calculamos el peso más frecuente para cada Description
frequent_weight = retail2_df.groupby('Description')['ProductWeight'].agg(lambda x: x.value_counts().idxmax())

# Luego, asignamos este peso más frecuente a cada Description en el dataframe original
retail2_df['ProductWeight'] = retail2_df['Description'].map(frequent_weight)

# Ordenar el DataFrame por 'Description'
retail2_df = retail2_df.sort_values(by='Description')

# Verificamos el resultado
# Mostrar las primeras filas de las columnas 'Description' y 'ProductWeight'
print(retail2_df[['Description', 'ProductWeight']].head(10))



                       Description  ProductWeight
352  ASSORTED COLOUR BIRD ORNAMENT           6.62
28   ASSORTED COLOUR BIRD ORNAMENT           6.62
257  ASSORTED COLOUR BIRD ORNAMENT           6.62
318  ASSORTED COLOUR BIRD ORNAMENT           6.62
360  ASSORTED COLOUR BIRD ORNAMENT           6.62
96   ASSORTED COLOUR BIRD ORNAMENT           6.62
189  ASSORTED COLOUR BIRD ORNAMENT           6.62
80   ASSORTED COLOUR BIRD ORNAMENT           6.62
167   BAKING SET 9 PIECE RETROSPOT           5.73
70    BAKING SET 9 PIECE RETROSPOT           5.73


In [1861]:
# Mostrar las categorías únicas en el dataframe
categorias = retail2_df['Category'].unique()

# Verificar el resultado
print("Categorías disponibles en el dataframe:")
print(categorias)

Categorías disponibles en el dataframe:
['Home & Garden' 'Toys' 'Beauty' 'Clothing']


In [1862]:
# Crear un diccionario para mapear las categorías a los proveedores
category_to_supplier = {
    'Home & Garden': 'Supplier A',
    'Toys': 'Supplier B',
    'Clothing': 'Supplier C',
    'Beauty': 'Supplier D'
}

# Asignar los proveedores a las categorías en el dataframe
retail2_df['Supplier'] = retail2_df['Category'].map(category_to_supplier)

# Ordenar el DataFrame por 'Description'
retail2_df = retail2_df.sort_values(by='Category')


# Mostrar las primeras filas de las columnas 'Description' y 'ProductWeight'
print(retail2_df[['Category', 'Supplier']].head(50))



     Category    Supplier
262    Beauty  Supplier D
278    Beauty  Supplier D
234    Beauty  Supplier D
270    Beauty  Supplier D
102    Beauty  Supplier D
301    Beauty  Supplier D
27     Beauty  Supplier D
19     Beauty  Supplier D
269    Beauty  Supplier D
290    Beauty  Supplier D
424    Beauty  Supplier D
38     Beauty  Supplier D
397    Beauty  Supplier D
314    Beauty  Supplier D
426    Beauty  Supplier D
125    Beauty  Supplier D
253    Beauty  Supplier D
21     Beauty  Supplier D
338    Beauty  Supplier D
14     Beauty  Supplier D
202    Beauty  Supplier D
265    Beauty  Supplier D
255    Beauty  Supplier D
67     Beauty  Supplier D
47     Beauty  Supplier D
299    Beauty  Supplier D
303    Beauty  Supplier D
187    Beauty  Supplier D
429    Beauty  Supplier D
75     Beauty  Supplier D
66     Beauty  Supplier D
11     Beauty  Supplier D
7      Beauty  Supplier D
108    Beauty  Supplier D
23     Beauty  Supplier D
436    Beauty  Supplier D
335  Clothing  Supplier C
372  Clothin

In [1863]:
# Exportar el dataframe a un archivo CSV con los cambios realizados
retail2_df.to_csv('retail2exp.csv', index=False, encoding='utf-8')